In [1]:
import pandas as pd
import numpy as np

In [2]:
index_for_df = ["p" + str(x) for x in range(9)]
index_for_df.append("next_move")
df = pd.read_csv('../games.csv').sample(frac=1)
df.columns = index_for_df
df.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,next_move
773,0,0,0,0,0,0,0,0,0,1
1238,0,0,0,0,0,0,0,0,0,2
5430,0,1,1,0,0,0,1,0,0,4
1481,0,1,0,0,0,-1,0,0,0,4
2348,0,0,0,0,0,0,0,0,0,1


In [3]:
def disclosure_to_vector(train:np.ndarray)-> np.ndarray:
    out = []
    for i in train:
        out.append(vector_representation(i, 9))
    return np.array(out, dtype=int)


def vector_representation(val, max_val:int=9) -> np.ndarray:
    out = np.zeros(max_val)
    out[val] = 1
    return out

In [4]:
y_train = disclosure_to_vector(df['next_move'].values)
x_train = df.drop(columns=['next_move']).values

In [5]:
y_train[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0])

In [10]:
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense;
import tensorflow as tf

In [7]:
weights_file = 'weights.h5'
callback = tf.keras.callbacks.ModelCheckpoint(weights_file,
              monitor='accuracy',
              mode='max',
              save_best_only=True)

In [9]:
model = Sequential()
model.add(Dense(units=20, activation='relu', input_dim=9))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=9, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train, epochs=100, batch_size=32, callbacks=[callback])

json_file = 'model.json'
model_json = model.to_json()

with open(json_file, 'w') as f:
  f.write(model_json)

Epoch 1/100
269/269 [==============================] - 1s 2ms/step - loss: 1.9324 - accuracy: 0.2492
Epoch 2/100
269/269 [==============================] - 0s 2ms/step - loss: 1.2947 - accuracy: 0.6572
Epoch 3/100
269/269 [==============================] - 0s 2ms/step - loss: 0.6694 - accuracy: 0.8572
Epoch 4/100
269/269 [==============================] - 0s 2ms/step - loss: 0.3653 - accuracy: 0.8572
Epoch 5/100
269/269 [==============================] - 0s 1ms/step - loss: 0.2686 - accuracy: 0.8536
Epoch 6/100
269/269 [==============================] - 0s 2ms/step - loss: 0.2356 - accuracy: 0.8564
Epoch 7/100
269/269 [==============================] - 0s 1ms/step - loss: 0.2217 - accuracy: 0.8552
Epoch 8/100
269/269 [==============================] - 0s 2ms/step - loss: 0.2147 - accuracy: 0.8530
Epoch 9/100
269/269 [==============================] - 0s 1ms/step - loss: 0.2105 - accuracy: 0.8558
Epoch 10/100
269/269 [==============================] - 0s 1ms/step - loss: 0.2078 - accura

In [14]:
pred_test = model.predict(x_train)

269/269 [==============================] - 0s 1ms/step


In [15]:
pred_y = [np.argmax(x) for x in pred_test]
print(sum(pred_y == df['next_move'].values) / len(pred_y))

0.8571927781013395


## Блок дообучения модели

In [11]:
json_file = 'model.json'
model_json = model.to_json()

with open(json_file, 'w') as f:
  f.write(model_json)

In [12]:
with open('model.json', 'r') as f:
  loaded_model = model_from_json(f.read())

In [13]:
loaded_model.load_weights('weights.h5')

In [14]:
test = np.array([x_train[0]])
predict = loaded_model.predict(test)
print(predict)

1/1 [==============================] - 0s 125ms/step
[[1.1314322e-04 4.7671276e-01 5.2240592e-01 1.9369263e-05 6.8292778e-05
  8.7450149e-05 4.3103297e-04 1.0686085e-04 5.5238444e-05]]


In [15]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [16]:
loaded_model.fit(x_train, y_train, epochs=100, batch_size=32, callbacks=[callback])

json_file = 'model.json'
model_json = model.to_json()

with open(json_file, 'w') as f:
  f.write(model_json)

Epoch 1/100
183/183 [==============================] - 1s 2ms/step - loss: 1.6017 - accuracy: 0.5880
Epoch 2/100
183/183 [==============================] - 0s 2ms/step - loss: 1.1903 - accuracy: 0.6106
Epoch 3/100
183/183 [==============================] - 0s 2ms/step - loss: 1.1120 - accuracy: 0.6133
Epoch 4/100
183/183 [==============================] - 0s 2ms/step - loss: 1.0753 - accuracy: 0.6121
Epoch 5/100
183/183 [==============================] - 0s 2ms/step - loss: 1.0503 - accuracy: 0.6155
Epoch 6/100
183/183 [==============================] - 0s 2ms/step - loss: 1.0300 - accuracy: 0.6178
Epoch 7/100
183/183 [==============================] - 0s 2ms/step - loss: 1.0121 - accuracy: 0.6188
Epoch 8/100
183/183 [==============================] - 0s 2ms/step - loss: 0.9962 - accuracy: 0.6220
Epoch 9/100
183/183 [==============================] - 0s 2ms/step - loss: 0.9807 - accuracy: 0.6246
Epoch 10/100
183/183 [==============================] - 0s 2ms/step - loss: 0.9671 - accura